<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/PostAnnotation/Notebooks/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers==4.28.0
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 12.5 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-b2ym0wod
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-b2ym0wod
  Resolved https://github.com/huggingface/accelerate to commit 5d558f21e27744491219002a375e158eee987e0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-

In [5]:
import pandas as pd
import numpy as np
import datasets

# Hugging Face

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Gdrive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/"

Mounted at /content/gdrive


# Dataset


In [6]:
train_ds = datasets.load_dataset("csv", data_files={"train": f"{dir}dataset/final/combined_train.csv"})
val_ds = datasets.load_dataset("csv", data_files={"validation": f"{dir}dataset/final/combined_validation.csv"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

## Prep

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator",
                                          do_lower_case=False)


In [10]:
#  pytorch
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
# Non k-fold
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,
    max_length=128)

train_tokenized = train_ds.map(preprocess_function, batched=True)
val_tokenized = val_ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/46262 [00:00<?, ? examples/s]

In [12]:
import evaluate

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)

    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(accuracy_metric.compute(predictions=preds, references = labels))
    return results

In [13]:
id2label, label2id = {},{}
for x in range(15):
    id2label[x] = x
        # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id[x] = x
        # {"NEGATIVE": 0, "POSITIVE": 1}

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer



model = AutoModelForSequenceClassification.from_pretrained("google/electra-base-discriminator",
                                                           num_labels=15,
                                                           id2label=id2label,
                                                           label2id=label2id
)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

# Training

In [ ]:
#Non-KFold
training_args = TrainingArguments(
    output_dir="Google/Electra-Thesis-NonKFold",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,                    #10
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_tokenized['train'], #mfc_tokenized["train"]
    eval_dataset=val_tokenized['validation'],  #mfc_tokenized["test"]
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()